################Topic generation:::Preprocess#################################################

In [1]:
#Get the data
import os
import importlib
import pandas as pd
oneonetwo_data = pd.read_csv("data"+os.sep+"part"+os.sep+"Suomi112_eng.csv", encoding='utf-8-sig')['content']
sos_live = pd.read_csv("data"+os.sep+"part"+os.sep+"SosLive_eng.csv", encoding='utf-8-sig')['content']

In [2]:
data = pd.concat([oneonetwo_data,sos_live])

In [3]:
import utils
importlib.reload(utils)
processor = utils.Processor()
processor.ini_dowload()

[nltk_data] Downloading package stopwords to data/lib/stopwords...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to data/lib/wordnet...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to data/lib/omw...


In [4]:
data = data.apply(lambda x: processor.preprocess(str(x)))
data = data.apply(lambda x: processor.tokenize(x))
data = data.apply(lambda x: processor.remove_stopwords(x))
data = data.apply(lambda x: processor.process_tokens(x))
data
data_copy = data

################Topic generation:::Topics#################################################
Mostly taken from https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

In [5]:
flat_data = [tag for sentence in data for tag in sentence ]
from gensim.corpora import Dictionary
dictionary = Dictionary(data)
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in data]

In [ ]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1

In [ ]:
bow_doc_4310 = bow_corpus[1]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0],dictionary[bow_doc_4310[i][0]],bow_doc_4310[i][1]))

In [ ]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

In [ ]:
#Topics with simple bag of words
import gensim
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

In [ ]:
# Topics with TFiDF vectorisation:
import gensim
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

In [ ]:
# Test on one document
document = bow_corpus[1]
for index, score in sorted(lda_model_tfidf[document], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))